## Import

In [1]:
import sys; sys.path.insert(0, '..')
import os
import time
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tabulate import tabulate
from dataclasses import dataclass
from joblib import load
from keras.models import Sequential, load_model
from keras.layers import Conv1D, ZeroPadding1D, AveragePooling1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.optimizers import RMSprop, Adam
from keras.losses import CategoricalCrossentropy
from keras import activations
from sklearn.metrics import confusion_matrix
from utils.df_utils import df_wrapper
from utils.declarations import training_files, testing_files, POSE_MAP

## Define data

In [2]:
AMOUNT_OF_SENSORS = 3
VALIDATION_TESTSET = "004"

## Load dataframes

In [3]:
pd.set_option('io.hdf.default.format', 'table')

x_train_arr = list()
for key in training_files:
    x_train_arr.append(pd.read_hdf(f"../../data/preprocessed_data/train_arr_{AMOUNT_OF_SENSORS}.h5", f"df{key}"))

x_test_dict = dict()
y_test_dict = dict()
for key in testing_files:
    x_test_dict[key] = pd.read_hdf(f"../../data/preprocessed_data/test_dict_{AMOUNT_OF_SENSORS}.h5", f"df{key}")
    y_test = x_test_dict[key]["Pose"]
    y_test.index = [i for i in range(len(y_test))]
    y_test_dict[key] = y_test

x_train = pd.concat([df.drop([' TimeStamp (s)', 'Pose'],axis=1) for df in x_train_arr])
y_train = pd.concat([df['Pose'] for df in x_train_arr])
x_test = x_test_dict[VALIDATION_TESTSET].drop([' TimeStamp (s)', 'Pose'], axis=1) 
y_test = y_test_dict[VALIDATION_TESTSET]
x_test_dict.pop(VALIDATION_TESTSET, None)


,TimeStamp (s),AccX (g),AccY (g),AccZ (g),GyroX (deg/s),GyroY (deg/s),GyroZ (deg/s),QuatW,QuatX,QuatY,...,GyroY (deg/s),GyroZ (deg/s),QuatW,QuatX,QuatY,QuatZ,LinAccX (g),LinAccY (g),LinAccZ (g),Pose
0,0.00,0.188324,-0.752145,-0.636368,0.326500,0.738324,0.474029,-0.902754,0.412072,0.118851,...,-0.019720,0.449070,-0.856174,0.302404,-0.120561,-0.401226,-0.003426,0.003405,0.005235,1
1,0.02,0.193812,-0.750303,-0.635945,1.344905,0.592093,0.258232,-0.902653,0.412238,0.119035,...,0.632029,0.415189,-0.856220,0.302425,-0.120482,-0.401136,-0.002061,0.003230,0.004595,1
2,0.04,0.195434,-0.753546,-0.636747,1.470452,0.479430,-0.070112,-0.902537,0.412469,0.119115,...,0.377054,0.138527,-0.856240,0.302393,-0.120382,-0.401148,0.000077,0.002129,0.005270,1
3,0.06,0.196129,-0.752597,-0.635481,1.077611,-0.001055,-0.275569,-0.902469,0.412618,0.119137,...,-0.019938,0.049091,-0.856239,0.302404,-0.120395,-0.401138,-0.002225,0.004453,0.009974,1
4,0.08,0.196556,-0.746188,-0.641954,-0.072016,-0.306926,-0.028435,-0.902678,0.412097,0.119413,...,0.243032,0.332257,-0.856243,0.302486,-0.120408,-0.401064,-0.004538,-0.000894,0.010130,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9410,188.20,0.186450,-0.777308,-0.607560,-1.495733,-2.149025,-0.842469,-0.882223,0.445263,0.030348,...,-0.808717,-0.377440,-0.762839,0.369573,-0.116014,-0.517719,-0.004429,0.004884,0.020385,2
9411,188.22,0.198456,-0.781783,-0.612723,-0.590164,-1.520824,-1.144590,-0.882252,0.445103,0.030453,...,2.332716,-0.057907,-0.762849,0.369831,-0.115723,-0.517585,0.031140,0.007705,0.001380,2
9412,188.24,0.184996,-0.783313,-0.604298,0.781578,1.258426,0.109423,-0.882190,0.445255,0.030636,...,0.848639,-1.900102,-0.762952,0.369401,-0.115445,-0.517803,0.037008,0.010702,0.005996,2
9413,188.26,0.173928,-0.776946,-0.600416,0.185493,0.584445,-0.039879,-0.882186,0.445265,0.030712,...,-2.616771,-1.677885,-0.762755,0.369199,-0.115698,-0.518180,0.012094,0.013374,0.022615,2


## Adjust dataframes

In [4]:
# Create numpy arrays
x_train_numpy = x_train.values
x_test_numpy = x_test.values
y_train_numpy = y_train.values
y_test_numpy = y_test.values

# Hot encode categories into numbers
y_train = to_categorical(y_train_numpy,9)
y_test = to_categorical(y_test_numpy,9)

## Precision

In [6]:
model_type: "ANN" or "CNN" or "LSTM" or "RFC" = "RFC"
ids = [1]
PRINT_CLASSIFICATION_MATRIX = False

for run_i in ids:
    if model_type != "RFC":
        model = load_model(f"../models/{model_type}_{AMOUNT_OF_SENSORS}_sensor{'er' if AMOUNT_OF_SENSORS > 1 else ''}_{run_i}.h5")
    else:
        model = load(f"../models/RFC_model_{AMOUNT_OF_SENSORS}.joblib")
    print("Classification accuracy:")
    classification_dict = dict()
    accuracy_list = list() 

    for key in x_test_dict:
        x_test_numpy = x_test_dict[key].drop([' TimeStamp (s)', 'Pose'], axis=1).values
        if model_type == "RFC": 
            x_test_numpy = x_test_dict[key].drop([' TimeStamp (s)', 'Pose'], axis=1)
        if model_type == "CNN":
            x_test_numpy = x_test_numpy.reshape(x_test_numpy.shape[0],x_test_numpy.shape[1], 1)
        classify = model.predict(x_test_numpy)
        classifications = [i.argmax() for i in classify]
        if model_type == 'RFC': classifications = classify
        annotated_positions = y_test_dict[key].to_numpy()
        correct_classifications = (classifications == annotated_positions).sum()
        accuracy_list.append(round(correct_classifications/len(classifications)*100,2))
        print(f"{key}: {accuracy_list[-1]}%")
        classification_dict[key] = classifications
    print(f"Average accuracy: {round(sum(accuracy_list)/len(accuracy_list),2)}%")
    

    if PRINT_CLASSIFICATION_MATRIX:
        width=16
        height=4*len(x_test_dict)
        fig, axes = plt.subplots(len(x_test_dict), 4, figsize=(width, height))
        for i, key in enumerate(x_test_dict):
            cols = [f"{key}: Model classifications", f"{key}: Model heatmap", f"{key}: Annotated classifications", f"{key}: Annotated heatmap"]
            df_predict = pd.DataFrame({' Timestamp (s)': x_test_dict[key][' TimeStamp (s)'],'Pose':classification_dict[key]})
            sns.lineplot(ax=axes[i, 0], data=df_predict,x=' Timestamp (s)',y='Pose')
            sns.heatmap(ax=axes[i, 1], data=confusion_matrix(y_test_dict[key], classification_dict[key]), cmap="YlGnBu", annot=True, fmt="d")
            sns.lineplot(ax=axes[i, 2], data=x_test_dict[key], x=" TimeStamp (s)", y='Pose')
            sns.heatmap(ax=axes[i, 3], data=confusion_matrix(y_test_dict[key], y_test_dict[key].to_numpy()), cmap="YlGnBu", annot=True, fmt="d")
            for ax, col in zip(axes[i], cols): ax.set_title(col)
        fig.tight_layout()
        plt.show()
        fig.savefig(f"{model_type}_plot.png")

Classification accuracy:
001: 94.93%
002: 76.92%
003: 89.29%
005: 78.08%
006: 85.91%
007: 85.22%
008: 44.51%
009: 88.49%
010: 82.25%
Average accuracy: 80.62%


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=59d486bc-e14d-4632-9064-12272fc72d11' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>